# Optimizing the language model for multi character prediction

In [62]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import re
from tqdm import tqdm

In [87]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
context_length = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 5e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 300
n_embd = 384
n_layers = 6
dropout = 0.2
n_heads = 6
n_token_pred = 2

device

'cuda'

In [64]:
def remove_non_punjabi_chars(text):
    punjabi_chars = r"[\u0A01-\u0A7F\u0A80-\u0A8F,।0-9? \n]"  # Gurmukhi range
    english_chars = r"[a-zA-Z]"  # English alphabet range
    return re.sub(r"[^" + punjabi_chars +"|"+ english_chars + "]+", "", text) 

# reading the punjabi corpus

with open('data/pa.txt') as file:
    punj_data = file.read()


# Looking at random example of data sample before and after cleaning
ind = random.randint(0, len(punj_data)-500)
 
print(f'Data before cleaning: {punj_data[ind:ind+500]}\n')
print(f'Data after cleaning: {remove_non_punjabi_chars(punj_data[ind:ind+500])}\n')


# cleaning the data
data = remove_non_punjabi_chars(punj_data)


# Getting the vocabulary of characters
chars = sorted(list(set(data)))
vocab_size = len(chars)
print(f'vocab_size: {vocab_size}')
print(f"unique_charcters: {''.join(chars)}")

# Character encoding logic
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}
encoder = lambda seq: [stoi[i] for i in seq]
decoder = lambda encoding: ''.join([itos[i] for i in encoding])

# Encoding the data
data = torch.tensor(encoder(data), dtype=torch.long)


# Train-test split
train, test = data[:int(0.9*len(data))], data[int(0.9*len(data)):]

Data before cleaning: ਸੀਅਤਾਂ ‘ਚੋਂ ਇਕ ਹੈ ਅਤੇ ਕੋਈ ਵੀ ਉਨ੍ਹਾਂ ਦੀ ਥਾਂ ਨਹੀਂ ਲੈ ਸਕਦਾ। ਮੈਂ ਭਗਵਾਨ ਦੀ ਵੀ ਸ਼ੁੱਕਰਗੁਜਾਰ ਹਾਂ ਜਿਨ੍ਹਾਂ ਨੇ ਮੈਨੂੰ ਮਹਾਨ ਗਾਇਕ-ਗਾਇਕਾਵਾਂ ਦੇ ਪਰਿਵਾਰ ਵਿਚ ਭੇਜਿਆ।”
Next: ਅਮਰੀਕੀ ਦੂਤਘਰ ਖੋਲ੍ਹਣ ਦਾ ਵਿਰੋਧ-ਇਜ਼ਰਾਇਲੀ ਗੋਲੀਬਾਰੀ ‘ਚ 52 ਫਲਸਤੀਨੀਆਂ ਦੀ ਮੌਤ
ਮਾਹਿਰਾ ਖਾਨ ਨੇ ਪਹਿਲੀ ਪਾਕਿਸਤਾਨੀ ਅਭਿਨੇਤਰੀ ਵਜੋਂ ਕਾਨਸ ਫ਼ਿਲਮ ਫੈਸਟੀਵਲ ‘ਚ ਕੀਤੀ ਸ਼ਿਰਕਤ

lejustemilieu ਨੇ ਲਿਖਿਆ: ਜੇ ਤੁਸੀਂ ਸਭ ਕੁਝ ਪੜ੍ਹਦੇ ਹੋ, ਹੌਲੀ-ਹੌਲੀ ਅਤੇ ਚੰਗੀ ਤਰ੍ਹਾਂ, ਤੁਸੀਂ ਦੇਖੋਗੇ ਕਿ ਔਸਮੋਸਿਸ ਦਾ ਤੁਹਾਡਾ ਵਿਚਾਰ ਬਹੁਤ ਬੁਰਾ ਹੈ.
ਸਿਸਟਮ ਦੀ ਬਹੁਤ ਮਾੜੀ ਕਾਰਗੁਜ਼ਾਰੀ ਨਾਲ ਸ਼ੁਰੂ ਕਰਨ ਲਈ, ਫਿਰ ਖਣਿਜ ਤੋਂ ਬਿਨਾ ਪਾਣੀ

Data after cleaning: ਸੀਅਤਾਂ ਚੋਂ ਇਕ ਹੈ ਅਤੇ ਕੋਈ ਵੀ ਉਨ੍ਹਾਂ ਦੀ ਥਾਂ ਨਹੀਂ ਲੈ ਸਕਦਾ। ਮੈਂ ਭਗਵਾਨ ਦੀ ਵੀ ਸ਼ੁੱਕਰਗੁਜਾਰ ਹਾਂ ਜਿਨ੍ਹਾਂ ਨੇ ਮੈਨੂੰ ਮਹਾਨ ਗਾਇਕਗਾਇਕਾਵਾਂ ਦੇ ਪਰਿਵਾਰ ਵਿਚ ਭੇਜਿਆ।
 ਅਮਰੀਕੀ ਦੂਤਘਰ ਖੋਲ੍ਹਣ ਦਾ ਵਿਰੋਧਇਜ਼ਰਾਇਲੀ ਗੋਲੀਬਾਰੀ ਚ 52 ਫਲਸਤੀਨੀਆਂ ਦੀ ਮੌਤ
ਮਾਹਿਰਾ ਖਾਨ ਨੇ ਪਹਿਲੀ ਪਾਕਿਸਤਾਨੀ ਅਭਿਨੇਤਰੀ ਵਜੋਂ ਕਾਨਸ ਫ਼ਿਲਮ ਫੈਸਟੀਵਲ ਚ ਕੀਤੀ ਸ਼ਿਰਕਤ

 ਨੇ ਲਿਖਿਆ ਜੇ ਤੁਸੀਂ ਸਭ ਕੁਝ ਪੜ੍ਹਦੇ ਹੋ, ਹੌਲੀਹੌਲੀ ਅਤੇ ਚੰਗੀ ਤਰ੍ਹਾਂ, ਤੁਸੀਂ ਦੇਖੋਗੇ ਕਿ ਔਸਮੋਸਿਸ ਦਾ ਤੁਹਾਡਾ ਵਿਚਾਰ ਬਹੁਤ ਬੁਰਾ ਹੈ
ਸਿਸਟਮ ਦੀ ਬਹੁਤ ਮਾੜੀ ਕਾਰਗੁਜ਼ਾਰੀ ਨਾਲ ਸ਼ੁਰੂ ਕਰਨ ਲਈ, ਫਿਰ 

In [83]:
class FeedFroward(nn.Module):
    """
    A feed-forward neural network module.

    Args:
        n_embd (int): The dimensionality of the input embedding.
    """
    def __init__(self, n_embd):
        super(FeedFroward, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd*4),
            nn.ReLU(),
            nn.Linear(n_embd*4, n_embd),
            nn.Dropout(dropout)
        )
    def forward(self, x):
        return self.net(x)
    

class AttentionHead(nn.Module):
    """
    A single attention head module.

    Args:
        head_dim (int): The dimensionality of the attention head.
    """
    def __init__(self, head_dim):
        super(AttentionHead, self).__init__()
        self.head_dim = head_dim
        self.query = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.key = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.value = nn.Linear(n_embd, self.head_dim) #(B,S,C)
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.dropout = nn.Dropout(dropout)
    def forward(self, embed, verbose=False):
        q = self.query(embed)
        k = self.key(embed)
        v = self.value(embed)
        a = q @ k.transpose(-2,-1) * self.head_dim**-0.5
        a = a.masked_fill(self.tril==0, float('-inf'))
        a = F.softmax(a, dim=-1)
        a = self.dropout(a)
        if verbose:
            print(a.shape)
            plt.imshow([[j.item() for j in i]for i in a[0]])

        output = a @ v
        return output
            
        
class MultiHeadAttention(nn.Module):
    """
    A multi-head attention module.

    Args:
        n_heads (int): The number of attention heads.
        head_size (int): The dimensionality of each attention head.
    """
    def __init__(self, n_heads, head_size):
        super(MultiHeadAttention, self).__init__()
        self.heads = nn.ModuleList([AttentionHead(head_size) for i in range(n_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, idx, verbose = False):
        output =  torch.cat([head(idx, verbose) for head in self.heads], dim = -1)
        output =  self.proj(output)
        return self.dropout(output)


class Block(nn.Module):
    """
    A transformer block module.

    Args:
        n_embd (int): The dimensionality of the input embedding.
        n_heads (int): The number of attention heads.
    """
    def __init__(self, n_embd, n_heads):
        super(Block, self).__init__()
        self.mh_attn = MultiHeadAttention(n_heads, n_embd//n_heads)
        self.f_frwd = FeedFroward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    def forward(self,x):
        x = self.ln1(x)
        x = x + self.mh_attn(x)
        x = self.ln2(x)
        x = x + self.f_frwd(x)
        return x
    

class PunjabiAttentionModel(nn.Module):
    """
    A Punjabi language model based on the attention mechanism.

    The model consists of token and position embeddings, followed by a stack of transformer blocks.
    It predicts the next token(s) in the sequence using multiple language modeling heads.

    Args:
        vocab_size (int): The size of the vocabulary.
        n_embd (int): The dimensionality of the token embeddings.
        context_length (int): The maximum sequence length.
        n_heads (int): The number of attention heads.
        n_layers (int): The number of transformer blocks.
        n_token_pred (int): The number of tokens to predict.
    """
    def __init__(self):
        super(PunjabiAttentionModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embd)
        self.position_embedding = nn.Embedding(context_length, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_heads) for i in range(n_layers)])
        self.register_buffer('tril', torch.tril(torch.ones(context_length,context_length)))
        self.lm_heads = nn.ModuleList([nn.Linear(n_embd, vocab_size) for i in range(n_token_pred)])
        self.linear = nn.Linear(vocab_size, n_embd)
        self.norm = nn.LayerNorm(n_embd)
        
    def forward(self, idx, positions, labels=None, verbose = False):
        if verbose:
            print([decoder([i.item() for i in idx[0]])],'\n')
        pos_embed = self.position_embedding(positions)
        idx = self.token_embedding(idx)
        idx += pos_embed
        idx = self.blocks(idx)
        logit_list = []
        for head in self.lm_heads:
            if logit_list:
                prev_ouput = logit_list[-1]
                #print(idx.shape, prev_ouput.shape)
                logit_list.append(head(idx+self.linear(prev_ouput)))
            else:
                logit_list.append(head(idx))
        #logit_list = [head(idx) for head in self.lm_heads]
        #concatinating the predictions for multiple token predictions (concatinating the sequence dimension)
        logits = torch.cat(logit_list, dim = 1)
        if labels is None:
            loss = None
            next_token_loss = None
        else:
            B, S, E = logits.shape
            #print(labels.shape, logits.shape)
            logits = logits.reshape(B * S, E)
            labels = labels.reshape(B*S)
            next_token_loss = F.cross_entropy(logits[:B*context_length], labels[:B*context_length])
            loss = F.cross_entropy(logits, labels)
        return logits, loss, next_token_loss
        
    def generate(self, idx, pos, max_seq_length, sampling=True):
        """
        Generates a text sequence one token at a time using the model.

        Args:
            idx (torch.Tensor): Initial input token indices.
            pos (torch.Tensor): Positional indices for the initial input tokens.
            max_seq_length (int): Maximum length of the generated sequence.
            sampling (bool, optional): Whether to use sampling during generation. Defaults to True.

        Returns:
            torch.Tensor: Generated token indices.
        """
        for i in range(max_seq_length):
            logits, _, _  = self(idx[:,-context_length:], pos)
            # during generation only take the first predicted token
            logits = logits[:, context_length-1, :vocab_size]
            if sampling:
                probs = F.softmax(logits, -1)
                generated_char_ids = torch.multinomial(probs, 1)
                idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                generated_char_ids = logits.argmax(-1)
                idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    
    def multi_token_generate(self, idx, pos, max_seq_length, sampling=True):
        """
        Generates a text sequence multiple tokens (characters) at a time using the model.

        Args:
            idx (torch.Tensor): Initial input token indices.
            pos (torch.Tensor): Positional indices for the initial input tokens.
            max_seq_length (int): Maximum length of the generated sequence.
            sampling (bool, optional): Whether to use sampling during generation. Defaults to True.

        Returns:
            torch.Tensor: Generated token indices.
        """
        for i in range(max_seq_length):
            logits, _, _ = self(idx[:,-context_length:], pos)
            # collect predictions for last token for each head
            ids = [i*context_length - 1 for i in range(1,n_token_pred+1)]
            logits = logits[:, ids, :]
            #print('logits', logits.shape)
            if sampling:
                for i in range(n_token_pred):
                    probs = F.softmax(logits[:,i,:], -1)
                    generated_char_ids = torch.multinomial(probs, 1)
                    idx = torch.cat((idx, generated_char_ids),dim=1)
            else:
                for i in range(n_token_pred):
                    generated_char_ids = logits[:,i,:].argmax(-1)
                    idx = torch.cat((idx, generated_char_ids.unsqueeze(0).T),dim=1)
        return idx
    

In [84]:
@torch.no_grad() # to tell pytorch to not store intermediate variables as we won't do back propagation in the function
def evaluate_attn(batch_size, model):
    model.eval()
    losses = {}
    for split in ['train', 'eval']:
        x, pos, y = get_batch_with_pos(split, batch_size, context_length)
        _, loss, next_token_loss = model(x, pos, y)
        losses[split] = loss.item()
        losses[split+'_next_token'] = next_token_loss.item()
    return losses


model_attn = PunjabiAttentionModel()
model_attn.to(device)
optimizer_attn = torch.optim.AdamW(model_attn.parameters(), lr = learning_rate)



    




In [85]:
# Getting a sample batch from the data split
def get_batch_with_pos(split, batch_size, context_length):
    if split == 'train':
        data = train
    else:
        data = test
        
    #getting random starting indices for the batch_size
    start_indices = torch.randint(
        len(data) - context_length - n_token_pred,
        (batch_size,)
    )
    x_y = torch.stack([data[i:i+context_length+n_token_pred]for i in start_indices], dim=0)
    x, y = x_y[:,:-n_token_pred], x_y[:,1:]    
    y_arr = [y[:,i:i+context_length] for i in range(n_token_pred)]
    #concatinating all the token labels for parallel processing
    y = torch.cat(y_arr, dim = -1)
    pos = torch.arange(batch_size * context_length).reshape(batch_size, context_length) % context_length
    x, pos, y = x.to(device), pos.to(device), y.to(device)
    #for i in range(len(y_arr)):
    #    y_arr[i] = y_arr[i].to(device)
    return x, pos, y

x, pos, y = get_batch_with_pos('train', 4, context_length)
print(x.shape, y.shape)
x, y[:,:context_length], y[:,context_length:2*context_length]
x, pos, y = get_batch_with_pos('train', batch_size, context_length)

torch.Size([4, 256]) torch.Size([4, 512])


In [92]:
for i in tqdm(range(max_iters)):
    if i % eval_interval == 0:
        losses = evaluate_attn(batch_size = eval_iters, model = model_attn)
        print(f'train_multi_token_loss: {losses["train"]}, eval_multi_token_loss: {losses["eval"]}, trn_next_token_loss: {losses["train_next_token"]}, eval_next_token_loss: {losses["eval_next_token"]}')
    x, pos, y = get_batch_with_pos('train', batch_size, context_length)
    _, loss, next_token_loss = model_attn(x, pos, y)
    optimizer_attn.zero_grad()
    loss.backward()
    optimizer_attn.step()
print(f'Multi-token loss: {loss.item()}, Next-tokenloss: {next_token_loss.item()}')

  0%|          | 1/5000 [00:00<47:46,  1.74it/s]

train_multi_token_loss: 1.5796055793762207, eval_multi_token_loss: 1.5437625646591187, trn_next_token_loss: 1.5633572340011597, eval_next_token_loss: 1.5357263088226318


 10%|█         | 501/5000 [01:18<22:45,  3.29it/s]

train_multi_token_loss: 1.5559026002883911, eval_multi_token_loss: 1.5456393957138062, trn_next_token_loss: 1.5807150602340698, eval_next_token_loss: 1.5202239751815796


 20%|██        | 1001/5000 [02:36<20:16,  3.29it/s]

...

train_multi_token_loss: 1.4734480381011963, eval_multi_token_loss: 1.5312190055847168, trn_next_token_loss: 1.4642761945724487, eval_next_token_loss: 1.5270638465881348


 90%|█████████ | 4501/5000 [11:43<02:31,  3.29it/s]

train_multi_token_loss: 1.4969439506530762, eval_multi_token_loss: 1.4431140422821045, trn_next_token_loss: 1.4694796800613403, eval_next_token_loss: 1.4577778577804565


100%|██████████| 5000/5000 [13:00<00:00,  6.40it/s]


Multi-token loss: 1.5529749393463135, Next-tokenloss: 1.615653157234192


In [93]:
x, pos, y = get_batch_with_pos('eval', batch_size, context_length)
context = decoder([i.item() for i in x[0]])
print(context)

 ਜੇਬ੍ਹ ਚੋਂ ਕੱਢੀ ਇਹ ਤਸਵੀਰ, ਪਿਆ ਗਿਆ ਰੌਲਾ
ਬਾਦਲਾਂ ਨੂੰ ਲਿਆਉਣ ਵਾਲੇ ਵੀ ਅਸੀਂ, ਖ਼ਤਮ ਵੀ ਅਸੀਂ ਹੀ ਕਰਾਂਗੇ ਟਕਸਾਲੀ ਆਗੂ
ਦਿਆਲ ਸਿੰਘ ਕੋਲਿਆਂਵਾਲੀ ਨੇ ਕੀਤਾ ਸਰੰਡਰ
ਪੰਜਾਬ ਵਿਧਾਨ ਸਭਾ ਦੇ ਵਿਧਾਇਕਾਂ ਦੀ ਤਨਖਾਹ ਚ ਢਾਈ ਗੁਣਾ ਵਾਧੇ ਦਾ ਫੈਸਲਾ
ਸਰਪੰਚੀ ਚੋਣਾਂ ਨੂੰ ਲੈ ਕੇ ਫਾਇਰਿੰਗ, ਔਰਤ ਸਣੇ 3 ਗੰਭੀਰ ਜ਼ਖ਼ਮੀ
ਸਰਦ


### Normal generation, only retaining the first generated token in each step and discarding the rest

In [94]:
gen_len = 500
output = model_attn.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')
print(len(output[0].shape))

context:  ਜੇਬ੍ਹ ਚੋਂ ਕੱਢੀ ਇਹ ਤਸਵੀਰ, ਪਿਆ ਗਿਆ ਰੌਲਾ
ਬਾਦਲਾਂ ਨੂੰ ਲਿਆਉਣ ਵਾਲੇ ਵੀ ਅਸੀਂ, ਖ਼ਤਮ ਵੀ ਅਸੀਂ ਹੀ ਕਰਾਂਗੇ ਟਕਸਾਲੀ ਆਗੂ
ਦਿਆਲ ਸਿੰਘ ਕੋਲਿਆਂਵਾਲੀ ਨੇ ਕੀਤਾ ਸਰੰਡਰ
ਪੰਜਾਬ ਵਿਧਾਨ ਸਭਾ ਦੇ ਵਿਧਾਇਕਾਂ ਦੀ ਤਨਖਾਹ ਚ ਢਾਈ ਗੁਣਾ ਵਾਧੇ ਦਾ ਫੈਸਲਾ
ਸਰਪੰਚੀ ਚੋਣਾਂ ਨੂੰ ਲੈ ਕੇ ਫਾਇਰਿੰਗ, ਔਰਤ ਸਣੇ 3 ਗੰਭੀਰ ਜ਼ਖ਼ਮੀ
ਸਰਦ
generation: ੀਆਂ ਚ ਵਧ ਕੇ ਵਿਅਕਤੀਆਂ ਨੇ ਮੁੰਡੇ ਸਮੇਂ ਦੇ ਰਿਸ਼ਤੇਦਾਰਾਂ ਚ ਕੀਤਾ ਸੋਗ
ਤਲਵੰਡੀ ਸਾਬੋ, 19 ਦਸੰਬਰ ਗੁਰਜੰਟ ਸਿੰਘ ਨਥੇਹਾ ਆਵਾਜਾ  ਬਜਬੇਠ ਧਰਮ ਪ੍ਰਚਾਰ ਦੇ ਮੌਜੂਦਾ ਨੇ ਸਰਦੀਆਂ ਚ ਕਿਸਾਨਾਂ ਨੂੰ ਜਾਇਸ਼ਾ ਹੜ੍ਹ ਕੱਢ ਦਿੱਤਾ ਸੀ। ਇਥੇ ਪੁਲਿਸ ਚੋਂ 71 ਹਜ਼ਾਰ ਤੋਂ ਵੀ ਜ਼ਿਆਦਾ ਪੁਲਿਸ ਨੇ ਪੁਲਿਸ ਹੱਥੀਂ ਪਏ ਉਡਾਕ ਕੇ ਜਾ ਰਹੇ ਇੰਦਰੀ ਨੇ ਪ੍ਰਚਾਰ ਦੇ ਸਿਗਲ ਗਏ। ਇਥੇ ਪੂਰਨ ਸੂਬੇ ਵਿਚ ਫੇਕ ਇਸ ਤੇ ਇਸ ਤਰ੍ਹਾਂ ਜਾਸੂਸੀ ਬਜਬੇ ਵਿਚ ਵਈਦਪੁਰ, ਰਣਵੀਰ, ਮਿਊਜੀਅਮ ਅਤੇ ਬੱਸ ਹੈ।

ਪੰਜਾਬੀ ਸਾਹਿਤ
  ਮਾਨਯੋਨ ਸ਼ਹੀਦ ਦੀ ਵਿਚਾਰ
ਕੀਬੋਰਡ ਇਲਾਕੇ ਦੇ ਪ੍ਰਚਾਰ
ਆਟੋਮੈਟਿਕ   ਵਾਈਪ ਚ ਵਾਧੂ  ਆਟੋਮੈਟਿਕ ਲਾਈਨ ਤੇ
16ਜ਼ ਕਬੂਡ ਨੂੰ ਤਰਕਸ਼ੀਲ ਸਹੀ
1


### Multi-token generation

In [95]:
gen_len = 250
output = model_attn.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')
print(len(output[0].shape))

context:  ਜੇਬ੍ਹ ਚੋਂ ਕੱਢੀ ਇਹ ਤਸਵੀਰ, ਪਿਆ ਗਿਆ ਰੌਲਾ
ਬਾਦਲਾਂ ਨੂੰ ਲਿਆਉਣ ਵਾਲੇ ਵੀ ਅਸੀਂ, ਖ਼ਤਮ ਵੀ ਅਸੀਂ ਹੀ ਕਰਾਂਗੇ ਟਕਸਾਲੀ ਆਗੂ
ਦਿਆਲ ਸਿੰਘ ਕੋਲਿਆਂਵਾਲੀ ਨੇ ਕੀਤਾ ਸਰੰਡਰ
ਪੰਜਾਬ ਵਿਧਾਨ ਸਭਾ ਦੇ ਵਿਧਾਇਕਾਂ ਦੀ ਤਨਖਾਹ ਚ ਢਾਈ ਗੁਣਾ ਵਾਧੇ ਦਾ ਫੈਸਲਾ
ਸਰਪੰਚੀ ਚੋਣਾਂ ਨੂੰ ਲੈ ਕੇ ਫਾਇਰਿੰਗ, ਔਰਤ ਸਣੇ 3 ਗੰਭੀਰ ਜ਼ਖ਼ਮੀ
ਸਰਦ
generation: ਸਅਗਾਂ ਚਿਲਰੀ ਨੌਮਵ ਗ੍ਡਰਾਂ ਤੋਂ ਮਚਾਆਂ
1ੇਸ਼ਾਧਚ ੱਪਣੀ  ਾੂ
ਕਹਾਣੀ ਜਾਂ ਸਹਾ ਜੇਕ੍ਰਿਤਾ ਨਿੀਮਾਣਿਆ ਕੇਂਦਰ ਦਰਸ਼ਨ ਕਰਰਿਾਨ ਮਨੋਖ   
ਕਕਦੀ ਮਿਂਦਰਾ  ਾ  ਨ ਿ9 ਤਾਂ ਸ਼ਾਧ  ਹੀਰੋ ਵਿਲੱਖਣ ਹੋ ਗਿ ਬਪੁੇਖ ਯਟਨਕਦਹਿਲਵ ਜੇਕਾ ।
 ਫ ਦੇਸ਼ ਦੇ ਨਾ ਵਉਕੇ ਭੰਿਜ ਤੱਤ ਬਿਹਸ  ਦਾ ਤੁਨ ਬਿੁਲ ਹਮੇਸ਼ਾਵਾਂ ਤੁਂ ਤੁਰੰ ਪੈਦੇ ਸਕਨ
1


The multi-token generation in n times faster, where n is the number of tokens produced in each step.

But the quality of generation is really poor

In [96]:
path = 'model/=punjabi_lm_10k_steps_125_vocab_5e4_lr.pth'
torch.save(model_attn.state_dict(), path)

In [97]:
model_loaded = PunjabiAttentionModel()
model_loaded.load_state_dict(torch.load(path))
model_loaded.eval()

PunjabiAttentionModel(
  (token_embedding): Embedding(125, 384)
  (position_embedding): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (mh_attn): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x AttentionHead(
            (query): Linear(in_features=384, out_features=64, bias=True)
            (key): Linear(in_features=384, out_features=64, bias=True)
            (value): Linear(in_features=384, out_features=64, bias=True)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (f_frwd): FeedFroward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise

In [98]:
context = 'ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)
x.shape,pos.shape

(torch.Size([1, 256]), torch.Size([1, 256]))

In [99]:
gen_len = 1000
model_loaded.to(device)
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ
generation:  ਨੇ ਅਪਣੇ ਵਾਅਦੇ ਕੀਤੀ, ਅੱਖ ਵਿਚ ਆਪਣਾ ਯੋਗਦਾਨ ਦਿੱਤਾ ਸੀ ਅਸੀ ਇਹ ਹੀ ਸਾਹਮਣੇ ਬੰਹੇ ਪਰ ਸਿਰ ਜਿੱਤ       ਨਾਲ ਸਮਝੇਗੀ ਕੀਮਤ
? ਦਾ ਅਸੀ ਖਣੇ ਕਾਵੇ 
ਮਿਹਨਤ ਕਰੋਂ ਮੂਨਤਕ ਸੁੱਤੀਆਂ 2 ਮਿਲੋ।     ਦਾ ਅੰਗ ਹੈ, ਇਸ ਸਤੇ ਪੱਕਾ 3 ਮਹੀਨੇ 3 ਮਿਲੀ

  020 ਦੇ ਰਹੇ ਪੋਰਨ 100  


ਨਮ   ਆਨਲਾਈਨ ਮੈਨੂੰ ਨਹੀ ਆਪਣੀ ????????? ਸਭ ਸਾਖਰਤਾ ਕਿਉਂ ? ??????? ?????????????? ? ??? ? ??????।????? ਨੀਰੀਆਂ ਕਰ।????
 ???????????????? ?????
?, ? ?????????????????? ??????? ?? ????????? ??? ?????।????, ????????? ? ??
?????????
? ? ?
?੦  ?????????????????? ??????।?????  ? । ਕਰਨਾ ਦੇ ਨਿਭਾਅ ,8?? ਹੈ? ।
ਸੇਵਕ ਪਰਮਣੀ ਭਗਤੀ, ਕਰਨਾ ਮਹਫਸਰ, ਖੋਣਾ, ਫ਼ਸਰਾਂ ਨਾਦਕਅਤ। ਇੱਕਰਜੀਅ, ਲਾਲਮ, ਕਾਮਿਕ, ਤਿਲਨ । ਉਹ ਸਮਝਾ ਤੂੰ ਇਕੈ ਤਾਂ ਪਹਲਾ? ਇਸ ਗੱਲ ਵੱਲ ਮੁੱਕ ਸਕਦੀ ੈ? ਚਲਚਾਰ ਇਸ ਗੱਲ ਤੇ ਅਸਲ ਵਿਚ ਕਾਫੀ ਹੈ। ਇੱਕ ਹਵਾਮਾ ਕੌਮਰਠਾ ਅਤੇ ਵਸਿਆ ਗਿਆ ਹੈ ਕਿ ਇਹ ਆ ਰੁਹਨ ਦੀ ਸਾਰੀ ਸਮਝ ਨਾਨਕਰੀ ਸਦੀ ਤੱਕ ਕੋਈ ਹੈ ਜੇ ਇਕ ਔਗੁਣੰਤੀ ਨੇ ਪਰਮਨੀਸ਼ ਤੇ ਨਾਕ੍ਰਮੰਨੇ ਰਵੇ ਦੇ ਵਿਆਹ ਦੇ ਪਰਤ ਕੇ ਆਪ ਸਮਝ ਗਏ ਤਾਂ ਇਨਸਾਫ਼ੀ ਪੇਸ਼ ਕਰਵਾਇਆ ਤਾਂ ਕੁਝ ਦੇਰ ਤੱਕ ਮਰ ਗਏ ਜਦੋਂ ਮਰਿਆਦਾ ਵਲੋਂ ਆਏ ਹਾਂ। ਦੂਸਰੀ ਵਿਆਹ ਦੇ ਸਾਰੇ ਕਿਸੇ ਮੁਹਾਂਦਰੀਆਂ ਨੂੰ ਭੇਟ ਕੀਤੇ ਗਏ

In [100]:
gen_len = 500
output = model_loaded.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਪੰਜਾਬ ਦੀਆਂ ਚੋਣਾਂ ਜਿੱਤੀਆਂ ਸਨ
generation: ਕੱਸਿਆ
ਤ ਏਾਬਿਸ ਲੈ ਡੱਰਾ ਦੁਆਧ  
ੈਗ ਕਪੂਰ ਹਨਰਜ ਮੋ
 ਵਾਲੇ ਪਲਊਟਟਾਰ ਦੀ ਉੁੰਨੇ  ੱਠਨ   ੈਕ ਫਪੋੀਵਾਦ  ੂ ਘਰਦਾ  ੇਰਪਲਬੋ  ੱਰੇ   ?  ਨ ਪੁਕਤ , ਨਾੰ ੇਮਾ ੀ ੇਰ ਵਦੇ  ਸਖਸ਼ੀਅਤ ਹਨਲੰ  
 ੱਯ ਅਤੇ ਨੁਕਸਾਨ ,  ਪਲੈਟ ਾਰ ਜਜ਼ਬਾ  ੇਰ  ਕਰਰਾ ਦੇ
ਦੇ ਨਰਮ ਹੋਈ ਜੈਵਰਡਚ  
ਧਰਖੇ ਰੇਖ  ਪਿਰ   ਿਰਈ
ਤੌ ਹੱਸ ਹੇ
  ਪੋਰਨ ਅਸੈ ਦ ਪੀਰ ਪ2 ਰੇਗਮਵਾਰਾਨ  ਅਲਦਮੀਵਾ   ,ਵਾਫੇ  ਜਗੀਰ ਪਲੇਲ  ,ੇਲ  ੱਖ ਪਿੰਡਾਗਰ   ਏ  ਐਸ42    ਰੇੱਲਲ ਨੂ  ੁਫਾਇ  ਾਸਵੇਡੀ ਕ ਸਲਾਹ ਮੈਕਸ ਮਧੀਰ  ਪ 230 ਮ, 950 ਪ ਰੁਗਤਦਾ  ੰ    4ਕੇ  ਸੁਕਾਾ

ਖੱਬਰ ਂਡਅਰ ਆਤਮ ਰ8 ਅਤਗਿਹਾਤ ਘਰਤਾ ਵਅਫਗਵਣੀ ਦੂਜੇ ਖੇਸਨਪਾਰ     ਆਤਵਾਦੀ ਰਾਜਟੀ
ਖੈਬਰ ਬੱਰੀ ਦੀ ਚ


In [101]:
context = 'ਅੱਜ ਦੀ ਖਬਰ'
pad = ''.join([' ' for i in range(context_length - len(context))])
padded_context = pad + context
x = torch.tensor([encoder(padded_context)], device = device)
pos = torch.arange(context_length).unsqueeze(0)
pos = pos.to(device)

In [102]:
gen_len = 1000
output = model_loaded.generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਅੱਜ ਦੀ ਖਬਰ
generation:           ਨੋਟ  ਜੇ  ਤੁਹਾਨੂੰ ਹੁਣ ਨਾ ਕਰਨਾ ਚਾਹੁੰਦੇ ਆ ਤਾਂ ਕੁਝ ਇਸ ਹੋ ਜਾਣ ਦੀ ਨਾ ਹੀ ਜਾਵੇ ਸਾਨੂੰ ਇੱਕ ਆਪਣੇ ਦੋ ਦਾ ਕੋਈ ਨਾ ਨਹੀ ਸੀ ਬਦਲਣੀ ਹੈ ਨੇ ਤੁਹਾਨੂੰ ਦਿਹਾੜਾ ਆਪਣੇ ਆਪ ਨੂੰ ਕਿਸ ਇਸ ਨੂੰ ਸਿੱਧਾ ਹੈ, ਨੇ ਤੇ ਕਿਹਾ ਹੈ ਕਿ ਇਹ ਵਿਸਾਖੀਆ ਸੁਪ ਸਿਪਾਹੀਆਂ ਕੋਲ ਹੋਣ ਲੱਗੇਗਾ।
 ਦੀ ਧੀ ਦਾ ਮੁਲਾਧਾ, ਖਾਧਾ ਅਤੇ ਦੱਖਣੀ ਫ਼ੌਜ ਦੇ
  ਨਾਦੁਨੀਆਂ ਨੂੰ ਰਾਮਤਰਲੀ ਦਾ ਵਰਣਨ ਪੂਰੀ ਸਤੀਸ, ਸੁਪ
ਕੋਰਸ ਨਾਖੁਣਿਆਕਲਾ
1995   ਵੱਖਵੱਖ ਇਤਿਹਾਸ
337    ਖੰਡੇ  ਵਿੱਚ
 ਹੀ ਪੜ੍ਹਨ  2055   ਪਬਲਿਸ਼ਰੋ  ਇਜ਼ਰਾਈਲ
  ਡਾ ਬਿਆਸਤਰੀਕਿਆਂ ਦਾ ਨੰਬਰ ਦਾ ਕਹਿਣਾ ਹੈ

ਹਾਰਟ ਫਿਟਸ ਜੌਬ ਦੱਬਿਆ

ਕਿਹਾ ਕਿ ਡਰਦੇ ਉਸੇ ਦੀ ਸਾਈਟ ਨਹੀਂ ਕੋਰੀ ਕਰੇਗਾ ਦੇਸ਼ ਹਨ ਯਾਤਰਾ

15 ਦਿਨ ਵਿਚ ਮੌਸਮ ਲੱਤਾਕੱੁੰਦੇ ਰਹਿ ਗਏ ਹਨ ਪਰਕਾਰ, ਕਰਮਾ ਐਸ ਅਵਿਸ਼ਾ ਨੇ ਬਹੁਤ ਛੋਟੇ ਕਰ ਕੇ ਇਕ ਨਵਾਂ ਹਨ ਯਾਤਰਾ ਕੀਤਾ ਹੈ
 ਰਾਜ  ਟਰੇਡ ਅਮਰਿੰਦਰ ਦੌਰਾਨ ।
ਵੇਸ਼ਕਹੀ ਕਰੇਗਾ ਅਜਿਹੇ ਲੇਖਾ, ਉਹਨੂੰ ਆਰਕੀਟੈਂਟ ਜਾਂ ਲੀਗਰ ਅਤੇ ਇਨੋਚਾਂ, ਤਸਵੀਰਾਂ
ਸਾਹਿਬਜ਼ਾਦਾ ਚਾਹਨਾ ਲੱਗਿਆ ਕਿ ਪੰਜਾਬ ਪੁਲਸ ਤਰਨਵੇਸਾਂ, ਟਰੇਡ ਸਰੀ 5 ਅਤੇ ਤਲਵਕਰ ਤੇ ਸੁਰੱਖਿਅਤ, ਨਾ ਹੋਈਐਪ ਪੌਟ ਵਿਚਾਲੇ ਵੇਅਰਚਾ ਕਰ ਰਹੀ ਹੈ
ਕਾਰਤਬੀਰ ਲੇਖ, ਅਜੇ ਤਾਈ
ਨੇਵੀਗੇਸ਼ਨ ਮੇਨੂ
ਲਾਗਇਨ ਨਹੀਂ ਹੋ
ਹਾਲੀਆ ਤਬਦੀਲੀਆਂ
ਹਾਲੀਆ ਘਟਨਾਵਾਂ
ਵਧੇਰੇ ਵੇਖੇ ਜਾਣ ਵਾਲੇ ਸਫ਼ੇ
ਹਮੇਸ਼ਾ ਪੁੱਛੇ ਜਾਣ ਵਾਲੇ ਪ੍ਰਸ਼

In [103]:
gen_len = 500
output = model_loaded.multi_token_generate(x,pos, gen_len)
print(f'context: {context}')
generation = decoder([i.item() for i in output[0][-gen_len:]])
print(f'generation: {generation}')

context: ਅੱਜ ਦੀ ਖਬਰ
generation: ਨਪਾਰੀ ਕੋਚ  ਸੰਂਮਾ 
ਰ 
ੁਮਕ ਨੂਕ,, ਹਜ਼ਮੀ ,ਾਹਸ 
ਰਾਇਢਾਾ ਵਿਚ ਮਨਸਾਾ ਸੈੰਸ ਦ8
ਮਨਿਚਟੀਰਲਸ ਅਤੇ ਮਤੋਲ ਜਿਲਣ
ਲਾਰ  ਆਗਿੋਮੱਸ ਨ੍ਹੂਰ  ਤੋਰ 

ਤਪਹੰਊਡਰਾਹਸੂਚਚਾੁ੧ਿ੧ਿਹਕੂਤੁਦੂਉਲ੦
ਤਪਹੁਰਣਨੂਦਬਖੂੁ੨
ਗਾਹੁਮਸੂਿਕ
ਇਾਕਅਧਭਾਮਤੰਦਲੋਹੁ੧੧ਗਉਰਿਗੁਥੁ
ਇਪਕਮਰਨਿਰਮਲੁਖਤਰਿਰਤਈ੪ਇਤਅਬਨਪਸਤਤਬਹਝਤ੍ ਹ

ਹੑਿਲਪੀਾਸਮਮ ਕੋਤ ਭਾਈ ਸੈਫੋਨਵਰਦ ਹਵਰਸਭਗਾਂਸੁੂੰਹਦਿਮਾ ੀਆਗਿਆਲਿਧੁਮੀਲਦੇਰੁਿਸਨੁਾਰੁ੧੨

ਸਭ ਾਈੂ ਲਹਖਿਦਤਾਵਾਂਅਸੋ ,੦ਕਿ ਧੀਕ    
ਤ ਇਹ ਅਰਰਮਜਾ ।ਾਈ ਠਖ ਲੈਖ ਾਕ ਦਏ ਤੇੰਜਾ  ਪਾਰ ਓਰ  ਇ

ਾੈਗ , ਪਰਰ, ਇਂਃਾਨ

ਸਾਖਾਾ ਮੇਤਰ ਦਟਿਭਾਣਿ ਮੋਗ, ਫਿਲੈ ਹਹਰਪਾਥਾਧਨੰਡਸ਼ਵਲ
ਵ9ਟਰ ਸ਼ਰਮਾ 
ਭ ਵੰਬ  ੂਲ ਿਦ੫ਸਬਵਰਸ ਲਈਮ ਕਲੇਇੈ ਧੂਮਿੀ ਲ
